# Chest X-ray Classification and Localization using DenseNet

This notebook implements a transfer learning approach using DenseNet for chest X-ray classification and localization on the NIH Chest X-ray dataset.

In [1]:
!pip install torch torchvision pandas numpy matplotlib tqdm scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 23.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 43.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 254.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 223.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 169.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 28.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 68.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 

In [234]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision import models
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

## Data Loading and Preprocessing

In [235]:
IMAGES_PATH_RESIZED = os.path.join('resized_images_2')
PREPROCESSED_IMAGES_PATH = os.path.join('bbox_resized_filtered_images_10k.csv')
df_preprocessed = pd.read_csv(PREPROCESSED_IMAGES_PATH)
df_preprocessed = df_preprocessed.sample(frac=1).reset_index(drop=True)

In [236]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class ChestXrayDataset(Dataset):
    def __init__(self, image_dir, df, transform=None):
        self.image_dir = image_dir
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['Image Index']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # Get labels (binary classification: normal vs abnormal)
        label = 1 if self.df.iloc[idx]['Finding Label'] != 'No Finding' else 0
        return image, torch.tensor(label, dtype=torch.float32)

Using device: cuda


In [237]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return > 0
print(torch.cuda.get_device_name(0))  # Should return the name of the GPU

True
1
NVIDIA A40


In [239]:
def get_transforms():
    """Define image transformations."""
    return {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

g = torch.Generator()
g.manual_seed(10)

# Split data and create data loaders
train_df, temp_df = train_test_split(df_preprocessed, test_size=0.2, random_state=10)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=10)

# Get transforms
transforms_dict = get_transforms()


In [240]:
# Create datasets
train_dataset = ChestXrayDataset(IMAGES_PATH_RESIZED, train_df, transform=transforms_dict['train'])
val_dataset = ChestXrayDataset(IMAGES_PATH_RESIZED, val_df, transform=transforms_dict['val'])
test_dataset = ChestXrayDataset(IMAGES_PATH_RESIZED, test_df, transform=transforms_dict['val'])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, generator=g)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")



Training samples: 7200
Validation samples: 900
Test samples: 900


## Model Training

In [262]:
class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        attn_map = self.sigmoid(self.conv(x))  # Generate attention map
        return x * attn_map, attn_map

class DenseNetWithAttention(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        # Load pre-trained DenseNet but freeze more layers
        densenet = models.densenet121(pretrained=True)
        self.features = densenet.features
        
        # 1) Freeze everything first
        for param in self.features.parameters():
            param.requires_grad = False

        # Unfreeze the last dense block (denseblock4) and norm5
        for name, child in self.features.named_children():
            if name == 'denseblock4' or name == 'norm5':
                for param in child.parameters():
                    param.requires_grad = True
        
        self.attention = SpatialAttention(1024)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Simpler classifier with stronger regularization
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),  # Added batch normalization
            nn.ReLU(),
            nn.Dropout(0.5), # Increased dropout
            nn.Linear(256, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # First run feature extraction
        x = self.features(x)
        # Then apply attention to features
        x, attn_map = self.attention(x)
        # Pool and classify
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        
        return logits, attn_map

# Create model
#model = DenseNetWithAttention().to(device)

In [227]:
class AttentionMemory:
    def __init__(self, memory_size=5, alpha=0.7):
        """
        Args:
            memory_size: Number of previous epochs to consider
            alpha: Exponential moving average factor (higher = more weight to recent)
        """
        #Keep the last 5 attention maps per image
        self.memory_size = memory_size
        #Controls how much more we trust recent maps (lower = longer memory)
        self.alpha = alpha
        self.attention_history = {}  # Maps image_id -> list of attention maps
        self.correct_predictions = {}  # Maps image_id -> list of correctness flags
    
    def update(self, image_ids, attention_maps, predictions, labels):
        """Update memory with new attention maps"""
        for i, img_id in enumerate(image_ids):
            # Get current attention map and whether prediction was correct
            attn = attention_maps[i].detach().cpu()
            correct = (predictions[i] > 0.5).float() == labels[i]
            
            # Initialize if first time seeing this image
            if img_id not in self.attention_history:
                self.attention_history[img_id] = []
                self.correct_predictions[img_id] = []
            
            # Add new data
            self.attention_history[img_id].append(attn)
            self.correct_predictions[img_id].append(correct.item())
            
            # Keep only most recent entries
            if len(self.attention_history[img_id]) > self.memory_size:
                self.attention_history[img_id].pop(0)
                self.correct_predictions[img_id].pop(0)
    
    def get_historical_attention(self, image_ids):
        """Get exponentially weighted historical attention maps"""
        batch_history = []
        
        for img_id in image_ids:
            if img_id not in self.attention_history or not self.attention_history[img_id]:
                # No history available
                batch_history.append(None)
                continue
            
            # Get history for this image
            history = self.attention_history[img_id]
            correctness = self.correct_predictions[img_id]
            
            if len(history) == 1:
                # Only one entry in history
                batch_history.append(history[0])
                continue
            
            # Calculate weighted average, giving more weight to:
            # 1. More recent attention maps
            # 2. Attention maps from correct predictions
            weights = []
            for i, is_correct in enumerate(correctness):
                # Position weight (more recent = higher weight)
                pos_weight = self.alpha ** (len(correctness) - i - 1)
                # Correctness weight (correct predictions get higher weight)
                correct_weight = 1.2 if is_correct else 0.8
                weights.append(pos_weight * correct_weight)
            
            # Normalize weights
            weights = [w / sum(weights) for w in weights]
            
            # Calculate weighted attention
            weighted_attn = torch.zeros_like(history[0])
            for i, attn in enumerate(history):
                weighted_attn += weights[i] * attn
            
            batch_history.append(weighted_attn)
        
        return batch_history

In [263]:
class SelfCorrectiveAttentionLoss(nn.Module):
    """
    Tell model how to correct its attention behaviour
    """
    def __init__(self, lambda_consistency=0.2, lambda_sparsity=0.1):
        """
        Args:
            lambda_consistency: Weight for consistency term
            lambda_sparsity: Weight for sparsity term
        """
        super(SelfCorrectiveAttentionLoss, self).__init__()
        self.cls_loss = nn.BCELoss() ## here maybe BCEWithLogitsLoss?
        self.lambda_consistency = lambda_consistency
        self.lambda_sparsity = lambda_sparsity
    
    def forward(self, pred, target, attn_map, historical_attn=None):

        """
            pred: model predictions
            target: ground truth labels
            attn_map: current attention map
            historical_attn: historical attention maps
        """
        # Classification loss (primary objective)
        cls_loss = self.cls_loss(pred, target)
        
        # Initialize attention losses
        consistency_loss = torch.tensor(0.0).to(pred.device)
        sparsity_loss = torch.tensor(0.0).to(pred.device)
        
        # Attention sparsity loss (encourage focused attention)
        # attention should be concentrated, not diffused
        attn_flat = attn_map.view(attn_map.size(0), -1)
        sparsity_loss = -torch.mean(torch.sum(attn_flat * torch.log(attn_flat + 1e-8), dim=1)) / 100
        
        # Consistency loss (if historical attention is available)
        if historical_attn is not None:
            valid_indices = [i for i, h in enumerate(historical_attn) if h is not None]
            
            if valid_indices:
                # Stack valid historical attention maps
                valid_hist = torch.stack([historical_attn[i] for i in valid_indices]).to(pred.device)
                valid_curr = attn_map[valid_indices]
                
                # Calculate consistency loss
                # For correct predictions: encourage consistency with history
                # For incorrect predictions: encourage deviation from history
                correct_mask = (pred.round() == target).float()[valid_indices].view(-1, 1, 1)
                incorrect_mask = 1 - correct_mask
                
                # For correct predictions: minimize distance to historical attention
                consistency_term = torch.abs(valid_curr - valid_hist) * correct_mask
                
                # For incorrect predictions: maximize distance to historical attention
                # But only if historical prediction was also wrong
                correction_term = torch.exp(-torch.abs(valid_curr - valid_hist)) * incorrect_mask
                
                consistency_loss = torch.mean(consistency_term + correction_term)
        
        # Total loss
        total_loss = cls_loss + self.lambda_consistency * consistency_loss + self.lambda_sparsity * sparsity_loss #(comment out for baseline classification loss)
        #total_loss = cls_loss
        
        return total_loss, cls_loss, consistency_loss, sparsity_loss

In [229]:
def visualize_attention_maps(model, data_loader, num_samples=5, save_dir='attention_maps'):
    """
    Visualize attention maps for a few samples
    
    Args:
        model: Trained model
        data_loader: DataLoader for the dataset
        num_samples: Number of samples to visualize
        save_dir: Directory to save visualizations
    """
    os.makedirs(save_dir, exist_ok=True)
    model.eval()
    
    # Get a few samples
    samples = []
    for images, labels, img_ids in data_loader:
        for i in range(min(len(images), num_samples - len(samples))):
            samples.append((images[i], labels[i], img_ids[i]))
        if len(samples) >= num_samples:
            break
    
    with torch.no_grad():
        for i, (image, label, img_id) in enumerate(samples):
            # Get model prediction and attention map
            image = image.unsqueeze(0).to(device)
            logit, attn_map = model(image)
            pred = torch.sigmoid(logit).item()
            
            # Convert to numpy for visualization
            image_np = image.squeeze().cpu().numpy().transpose(1, 2, 0)
            # Denormalize image
            image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            image_np = np.clip(image_np, 0, 1)
            
            # Get attention map
            attn_map_np = attn_map.squeeze().cpu().numpy()
            
            # Create visualization
            plt.figure(figsize=(12, 5))
            
            # Original image
            plt.subplot(1, 3, 1)
            plt.imshow(image_np)
            plt.title(f"Original Image\nTrue: {label.item()}")
            plt.axis('off')
            
            # Attention map
            plt.subplot(1, 3, 2)
            plt.imshow(attn_map_np, cmap='hot')
            plt.title(f"Attention Map\nPred: {pred:.4f}")
            plt.axis('off')
            
            # Overlay
            plt.subplot(1, 3, 3)
            plt.imshow(image_np)
            plt.imshow(attn_map_np, cmap='hot', alpha=0.5)
            plt.title("Overlay")
            plt.axis('off')
            
            plt.tight_layout()
            plt.savefig(f"{save_dir}/attention_{img_id.replace('.', '_')}.png")
            plt.close()

In [264]:
from torch.optim.lr_scheduler import CosineAnnealingLR

def train_with_self_correction(model, train_loader, val_loader, num_epochs=15):
    """
    Train the model with self-corrective attention 
    """
    # Initialize attention memory
    memory = AttentionMemory(memory_size=3, alpha=0.7)
    
    # Loss and optimizer
    criterion = SelfCorrectiveAttentionLoss(lambda_consistency=0.001, lambda_sparsity=0.001)

    #optimizer = optim.AdamW([
    #    #{'params': model.features.denseblock4.parameters(), 'lr': 5e-5},
    #    {'params': model.features.norm5.parameters(), 'lr': 5e-5},
    #    {'params': model.attention.parameters(), 'lr': 1e-4},
    #    {'params': model.classifier.parameters(), 'lr': 5e-3}
    #], weight_decay=1e-4)  # Increased weight decay

    #optimizer = optim.SGD([
    #    {'params': model.features.norm5.parameters(), 'lr': 1e-4},
    #    {'params': model.attention.parameters(), 'lr': 5e-4},
    #    {'params': model.classifier.parameters(), 'lr': 1e-3}
    #], momentum=0.9, weight_decay=1e-4)

    # Use different learning rates for different parts of the model
    base_params = [p for n, p in model.named_parameters() if 'features' not in n]
    feature_params = [p for n, p in model.named_parameters() if 'features' in n and p.requires_grad]

    optimizer = torch.optim.AdamW([
        {'params': base_params},
        {'params': feature_params, 'lr': 1e-5}  # Lower learning rate for pre-trained layers
    ], lr=5e-4, weight_decay=1e-4)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, verbose=True
    )

        
    # Track metrics
    best_val_loss = float('inf')
    history = {
        'train_loss': [], 'train_acc': [], 'train_auc': [],
        'val_loss': [], 'val_acc': [], 'val_auc': [],
        'cls_loss': [], 'consistency_loss': [], 'sparsity_loss': []
    }
    
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        epoch_cls_loss = 0.0
        epoch_consistency_loss = 0.0
        epoch_sparsity_loss = 0.0
        
        # For AUC calculation
        train_preds = []
        train_labels = []
        
        for images, labels, img_ids in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images = images.to(device)
            labels = labels.to(device)
            
            # Get historical attention maps for batch
            historical_attn = memory.get_historical_attention(img_ids)
            
            # Forward pass
            optimizer.zero_grad()
            logits, attn_maps = model(images)
            logits = logits.squeeze()
            
            # Calculate loss
            loss, cls_loss, consistency_loss, sparsity_loss = criterion(
                logits, labels, attn_maps, historical_attn
            )
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Track metrics
            train_loss += loss.item()
            epoch_cls_loss += cls_loss.item()
            epoch_consistency_loss += consistency_loss.item()
            epoch_sparsity_loss += sparsity_loss.item()
            
            # Calculate accuracy
            preds = (logits > 0.5).float()
            train_correct += (preds == labels).sum().item()

            # Store predictions and labels for AUC
            train_preds.extend(logits.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
            
            # Update attention memory
            memory.update(img_ids, attn_maps, logits, labels)

        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0

        # For AUC calculation
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for images, labels, img_ids in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                # Get historical attention
                historical_attn = memory.get_historical_attention(img_ids)
                
                # Forward pass
                logits, attn_maps = model(images)
                logits = logits.squeeze()
                
                # Calculate loss
                loss, _, _, _ = criterion(logits, labels, attn_maps, historical_attn)
                
                # Track metrics
                val_loss += loss.item()
                preds = (logits > 0.5).float()
                val_correct += (preds == labels).sum().item()

                # Store predictions and labels for AUC
                val_preds.extend(logits.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                
                # Update memory even for validation (helps track consistency)
                #memory.update(img_ids, attn_maps, logits, labels)
        
        # Calculate epoch metrics
        avg_train_loss = train_loss / len(train_loader)
        avg_cls_loss = epoch_cls_loss / len(train_loader)
        avg_consistency_loss = epoch_consistency_loss / len(train_loader)
        avg_sparsity_loss = epoch_sparsity_loss / len(train_loader)
        train_accuracy = 100 * train_correct / len(train_loader.dataset)
        train_auc = roc_auc_score(train_labels, train_preds)
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / len(val_loader.dataset)
        val_auc = roc_auc_score(val_labels, val_preds)
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['train_auc'].append(train_auc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        history['val_auc'].append(val_auc)
        history['cls_loss'].append(avg_cls_loss)
        history['consistency_loss'].append(avg_consistency_loss)
        history['sparsity_loss'].append(avg_sparsity_loss)
        
        # Print metrics
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {avg_train_loss:.4f} (Cls: {avg_cls_loss:.4f}, ' + 
              f'Consistency: {avg_consistency_loss:.4f}, Sparsity: {avg_sparsity_loss:.4f})')
        print(f'Train Accuracy: {train_accuracy:.2f}%, AUC: {train_auc:.4f}')
        print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%, AUC: {val_auc:.4f}')
        
        # Update scheduler
        scheduler.step(avg_val_loss)
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model_self_corrective.pth')
            print('Model saved!')
        
        print('-' * 60)

        #if epoch % 2 == 0:  # Save every 2 epochs to reduce storage
        #    print("Visualizing attention maps...")
        #    visualize_attention_maps(model, train_loader, 
        #                            save_dir=f'attention_maps/epoch_{epoch+1}/train')
        #    visualize_attention_maps(model, val_loader, 
        #                            save_dir=f'attention_maps/epoch_{epoch+1}/val')

    
    plt.figure(figsize=(15, 10))
    
    # Plot 1: Loss curves
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot 2: Accuracy curves
    plt.subplot(2, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.ylim(50, 100)
    plt.legend()
    
    # Plot 3: AUC curves
    plt.subplot(2, 2, 3)
    plt.plot(history['train_auc'], label='Train AUC')
    plt.plot(history['val_auc'], label='Validation AUC')
    plt.title('AUC Curves')
    plt.xlabel('Epoch')
    plt.ylabel('AUC Score')
    plt.ylim(0.5, 1)
    plt.legend()
    
    # Plot 4: Component losses
    plt.subplot(2, 2, 4)
    plt.plot(history['cls_loss'], label='Classification Loss')
    plt.plot(history['consistency_loss'], label='Consistency Loss')
    plt.plot(history['sparsity_loss'], label='Sparsity Loss')
    plt.title('Component Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.ylim(0, 0.7)
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.close()
    
    return model, history

In [267]:
class ChestXrayDatasetWithIDs(Dataset):
    def __init__(self, image_dir, df, transform=None):
        self.image_dir = image_dir
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['Image Index']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # Get labels (binary classification: normal vs abnormal)
        label = 1 if self.df.iloc[idx]['Finding Label'] != 'No Finding' else 0
        
        # Return image ID along with image and label
        return image, torch.tensor(label, dtype=torch.float32), img_name

# Create datasets with image IDs
train_dataset = ChestXrayDatasetWithIDs(IMAGES_PATH_RESIZED, train_df, transform=transforms_dict['train'])
val_dataset = ChestXrayDatasetWithIDs(IMAGES_PATH_RESIZED, val_df, transform=transforms_dict['val'])
test_dataset = ChestXrayDatasetWithIDs(IMAGES_PATH_RESIZED, test_df, transform=transforms_dict['val'])

batch_size = 32
# Create data loaders
g = torch.Generator()
g.manual_seed(10)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, generator=g, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 7200
Validation samples: 900
Test samples: 900


In [268]:
# Create model
model = DenseNetWithAttention().to(device)

# Train with self-correction
trained_model, history = train_with_self_correction(model, train_loader, val_loader, num_epochs=15)

/root/XAI_medical_deeplearning/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/XAI_medical_deeplearning/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/root/XAI_medical_deeplearning/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/15: 100%|██████████| 225/225 [00:13<00:00, 17.06it/s]


Epoch 1/15:
Train Loss: 0.5391 (Cls: 0.5189, Consistency: 0.0516, Sparsity: 0.1500)
Train Accuracy: 75.82%, AUC: 0.6755
Validation Loss: 0.5055, Accuracy: 78.89%, AUC: 0.7334
Model saved!
------------------------------------------------------------


Epoch 2/15: 100%|██████████| 225/225 [00:14<00:00, 15.74it/s]


Epoch 2/15:
Train Loss: 0.4990 (Cls: 0.4557, Consistency: 0.2950, Sparsity: 0.1382)
Train Accuracy: 79.07%, AUC: 0.7644
Validation Loss: 0.4824, Accuracy: 79.89%, AUC: 0.7493
Model saved!
------------------------------------------------------------


Epoch 3/15: 100%|██████████| 225/225 [00:13<00:00, 17.24it/s]


...

Epoch 13/15:
Train Loss: 0.2649 (Cls: 0.2378, Consistency: 0.1774, Sparsity: 0.0936)
Train Accuracy: 90.21%, AUC: 0.9502
Validation Loss: 0.5751, Accuracy: 79.00%, AUC: 0.7635
------------------------------------------------------------


Epoch 14/15: 100%|██████████| 225/225 [00:13<00:00, 16.29it/s]


Epoch 14/15:
Train Loss: 0.2492 (Cls: 0.2234, Consistency: 0.1691, Sparsity: 0.0889)
Train Accuracy: 91.11%, AUC: 0.9559
Validation Loss: 0.5872, Accuracy: 78.33%, AUC: 0.7618
------------------------------------------------------------


Epoch 15/15: 100%|██████████| 225/225 [00:14<00:00, 15.20it/s]


Epoch 15/15:
Train Loss: 0.2377 (Cls: 0.2125, Consistency: 0.1644, Sparsity: 0.0871)
Train Accuracy: 91.62%, AUC: 0.9607
Validation Loss: 0.6212, Accuracy: 78.89%, AUC: 0.7569
------------------------------------------------------------


In [12]:
memory = AttentionMemory(memory_size=5, alpha=0.7)